In [1]:
# import modules
import numpy as np
from sklearn.datasets import load_digits
from abc import abstractmethod

# Base classes

%%html
<div style="color: green; font-weight:bold"> The corresponding functions have been implemented correctly. But just as a reminder some constraints for thresholds finding function could be added to make the result accurate, however, for this dataset it doesn't hold.</div>

In [2]:
class Node:
    '''
      this class will later get the following attributes
      all nodes:
          features
          responses
      split nodes additionally:
          left
          right
          split_index
          threshold
      leaf nodes additionally
          prediction
    '''

class Tree:
    '''
      base class for RegressionTree and ClassificationTree
    '''
    def __init__(self, n_min=10):
        '''n_min: minimum required number of instances in leaf nodes
        '''
        self.n_min = n_min 
    
    def predict(self, x):
        ''' return the prediction for the given 1-D feature vector x
        '''
        # first find the leaf containing the 1-D feature vector x
        node = self.root
        while not hasattr(node, "prediction"):
            j = node.split_index 
            if x[j] <= node.threshold:
                node = node.left
            else:
                node = node.right
        # finally, return the leaf's prediction
        return node.prediction
        
    def train(self, features, responses, D_try=None):
        '''
        features: the feature matrix of the training set
        response: the vector of responses
        '''
        N, D = features.shape
        assert(responses.shape[0] == N)

        if D_try is None:
            D_try = int(np.sqrt(D)) # number of features to consider for each split decision
        
        # initialize the root node
        self.root = Node()
        self.root.features  = features
        self.root.responses = responses

        # build the tree
        stack = [self.root]
        while len(stack):
            node = stack.pop()
            active_indices = self.select_active_indices(D, D_try)
            left, right = self.make_split_node(node, active_indices)
            if left is None: # no split found
                self.make_leaf_node(node)
            else:
                stack.append(left)
                stack.append(right)
    
    def make_split_node(self, node, indices):
        '''
        node: the node to be split
        indices: a numpy array of length 'D_try', containing the feature 
                         indices to be considered for the present split
                         
        return: None, None -- if no suitable split has been found, or
                left, right -- the children of the split
        '''
        # all responses equal => no improvement possible by any split
        if np.unique(node.responses).shape[0] == 1:
            return None, None
        
        # find best feature j_min (among 'indices') and best threshold t_min for the split
        l_min = float('inf')  # upper bound for the loss, later the loss of the best split
        j_min, t_min = None, None

        for j in indices:
            thresholds = self.find_thresholds(node, j)

            # compute loss for each threshold
            for t in thresholds:
                loss = self.compute_loss_for_split(node, j, t)

                # remember the best split so far 
                # (the condition is never True when loss = float('inf') )
                if loss < l_min:
                    l_min = loss
                    j_min = j
                    t_min = t

        if j_min is None: # no split found
            return None, None

        # create children for the best split
        left, right = self.make_children(node, j_min, t_min)

        # turn the current 'node' into a split node
        # (store children and split condition)
        # my code 
        node.left = left
        node.right = right
        node.split_index = j_min
        node.threshold = t_min

        # return the children (to be placed on the stack)
        return left, right
    
    def select_active_indices(self, D, D_try):
        ''' return a 1-D array with D_try randomly selected indices from 0...(D-1).
        '''
        # my code
        rand_idx = np.arange(D)
        np.random.shuffle(rand_idx)
        return rand_idx[:D_try]
        
        
    def find_thresholds(self, node, j):
        ''' return: a 1-D array with all possible thresholds along feature j
        '''
        # my code
        sorted = np.sort(node.features[:,j])
        thresholds = (sorted[:-1]+ sorted[1:]) /2
        # sorts the jth coulumn according to it's value. The threshold is computed as the inbetween values of the sorted values.
        return thresholds


    def make_children(self, node, j, t):
        ''' execute the split in feature j at threshold t
        
            return: left, right -- the children of the split, with features and responses
                                   properly assigned according to the split
        '''
        left = Node()
        right = Node()

        # my code
        mask_left = node.features[:,j] <= t
        mask_right = node.features[:,j] > t
        
        left.features = node.features[mask_left]
        left.responses = node.responses[mask_left]

        right.features = node.features[mask_right]
        right.responses = node.responses[mask_right]

        return left, right
        
    @abstractmethod
    def make_leaf_node(self, node):
        ''' Turn node into a leaf by computing and setting `node.prediction`
        
            (must be implemented in a subclass)
        '''
        raise NotImplementedError("make_leaf_node() must be implemented in a subclass.")
        
    @abstractmethod
    def compute_loss_for_split(self, node, j, t):
        ''' Return the resulting loss when the data are split along feature j at threshold t.
            If the split is not admissible, return float('inf').
        
            (must be implemented in a subclass)
        '''

# Regression Tree

%%html
<div style="color: green; font-weight:bold"> Regression tree is implemented correctly, however, instead of calling the len and sum functions and doing some algebra, you could have used mean function  </div>

In [3]:
class RegressionTree(Tree):
    def __init__(self, n_min=1):
        super(RegressionTree, self).__init__(n_min)
        
    def compute_loss_for_split(self, node, j, t):
        # return the loss if we would split the instance along feature j at threshold t
        # or float('inf') if there is no feasible split
        # my code
        mask_left = node.features[:,j] <= t
        mask_right = node.features[:,j] > t
        
       
        responses_left = node.responses[mask_left]
        responses_right = node.responses[mask_right]
        
        if len(responses_left) < 10 or len(responses_right) < 10: 
            return float('inf')
        
        prediction_left = 1/len(responses_left) * np.sum(responses_left)
        prediction_right = 1/len(responses_right) * np.sum(responses_right)
        loss = np.sum((responses_left-prediction_left)**2) + np.sum((responses_right-prediction_right)**2)
        return loss 

    def make_leaf_node(self, node):
        # turn node into a leaf node by computing `node.prediction`
        # (note: the prediction of a regression tree is a real number)
        # my code
        node.prediction = np.mean(node.responses)


# Classification Tree

%%html
<div style="color: green; font-weight:bold"> the entropy loss is implemented wringly  </div>

In [4]:
class ClassificationTree(Tree):
    '''implement classification tree so that it can handle arbitrary many classes
    '''
    
    def __init__(self, classes, n_min=10):
        ''' classes: a 1-D array with the permitted class labels
            n_min: minimum required number of instances in leaf nodes
        '''
        super(ClassificationTree, self).__init__(n_min)
        self.classes = classes
        
    def compute_loss_for_split(self, node, j, t):
        # return the loss if we would split the instance along feature j at threshold t
        # or float('inf') if there is no feasible split
        mask_left = node.features[:,j] <= t
        mask_right = node.features[:,j] > t

        responses_left = node.responses[mask_left]
        responses_right = node.responses[mask_right]
        
        if len(responses_left) < 10 or len(responses_right) < 10:
            return float('inf')
        
        n_left_k = np.array([])
        n_right_k = np.array([])

        for k in np.unique(responses_left):
            mask = responses_left == k
            np.append(n_left_k,np.sum(mask))
        for k in np.unique(responses_right):
            mask = responses_right == k
            np.append(n_right_k,np.sum(mask))
        
        N_left = len(responses_left)
        N_right = len(responses_right)

        p_left_k = n_left_k/N_left
        p_right_k = n_right_k/N_right

        g_loss = N_left *(1-np.sum(p_left_k**2)) + N_right *(1-np.sum(p_right_k**2))
        return g_loss

    def make_leaf_node(self, node):
        # turn node into a leaf node by computing `node.prediction`
        # (note: the prediction of a classification tree is a class label)
        # my code
        unique_values, counts = np.unique(node.responses, return_counts=True)
        max_count_index = np.argmax(counts)
        most_frequent_response = unique_values[max_count_index]
        node.prediction = most_frequent_response

# Evaluation of Regression and Classification Tree

%%html
<div style="color: green; font-weight:bold"> The following is correct </div>

In [5]:
# read and prepare the digits data and extract 3s and 9s
digits = load_digits()
print(digits.data.shape, digits.target.shape)

instances = (digits.target == 3) | (digits.target == 9) 
features_39 = digits.data[instances, :]
labels_39 = digits.target[instances]

# for regression, we use labels +1 and -1
responses_39 = np.array([1 if l == 3 else -1 for l in labels_39])

assert(features_39.shape[0] == labels_39.shape[0] == responses_39.shape[0])

(1797, 64) (1797,)


%%html
<div style="color: green; font-weight:bold"> It is supposed to calculate the mean of non-matching predictions with the actual responses, therefore, using the difference of them resulted to the wrong solution and the similarity of cross validation for regression and classification could not be reached. </div>

In [6]:
# perform 5-fold cross-validation (see ex01) with responses +1 and -1 (for 3s and 9s)
# using RegressionTree()
# and comment on your results
# my code
from sklearn.model_selection import KFold

def cross_validation_regression(model, features, responses, num_sample):
    #as the metric we use MAE
    absolute_error = np.zeros(num_sample)
    k_folds = KFold(n_splits= num_sample)
    
    for i, (train, test) in enumerate(k_folds.split(features)):
        model.train(features[train], responses[train])
        predictions = np.array([])
        
        for f in features[test]:
            predictions = np.append(predictions, model.predict(f)) 
            
        absolute_error[i] = 1/(len(predictions)) * np.sum(np.abs(predictions - responses[test]))
        
    mean_absolute_error = np.mean(absolute_error)
    mean_absolute_error_std = np.std(absolute_error)
    print('Mean absolute error: ', mean_absolute_error, '+- ', mean_absolute_error_std)

cross_validation_regression(RegressionTree(n_min= 10), features_39, responses_39, 5)

Mean absolute error:  0.2339428322092981 +-  0.07486074312618579


In [7]:
# perform 5-fold cross-validation with labels 3 and 9
# using ClassificationTree(classes=np.unique(labels))
# and comment on your results
# my code
def cross_validation_classification(model, features, labels, num_sample):
    error_rate = np.zeros(num_sample)
    k_folds = KFold(n_splits= num_sample)
    
    for i, (train, test) in enumerate(k_folds.split(features)):
        model.train(features[train], labels[train])
        predictions = np.array([])
        
        for f in features[test]:
            predictions = np.append(predictions, model.predict(f))
    
        error_rate[i] = np.mean(predictions != labels[test])
    mean_error_rate = np.mean(error_rate)
    std_error_rate = np.std(error_rate)
    print('Mean error rate: ', mean_error_rate, '+-', std_error_rate)
    
cross_validation_classification(ClassificationTree(classes= np.unique(labels_39), n_min= 10), features_39, labels_39, 5)

Mean error rate:  0.25022831050228306 +- 0.08071622104844442


The mean error rate fluctuates between (20-40)% since we use different metrics for classification and regression, we can't really compare both results. 

# Regression and Classification Forest

%%html
<div style="color: green; font-weight:bold"> This section is implemented correctly</div>

In [8]:
def bootstrap_sampling(features, responses):
    '''return a bootstrap sample of features and responses
    '''
    # my code
    # N = features.shape[0]
    N = len(features)
    i = np.random.randint(0, N, size=N)
    return features[i], responses[i]

In [9]:
class RegressionForest():
    def __init__(self, n_trees, n_min=10):
        # create ensemble
        self.trees = [RegressionTree(n_min) for i in range(n_trees)]
    
    def train(self, features, responses):
        for tree in self.trees:
            boostrap_features, bootstrap_responses = bootstrap_sampling(features, responses)
            tree.train(boostrap_features, bootstrap_responses)

    def predict(self, x):
        # compute the response of the ensemble from the individual responses and return it
        # my code
        predictions = np.array([])
        
        for tree in self.trees:
            predictions = np.append(predictions, tree.predict(x))
        
        predictions = np.mean(predictions)
        return predictions

In [10]:
class ClassificationForest():
    def __init__(self, n_trees, classes, n_min=1):
        self.trees = [ClassificationTree(classes, n_min) for i in range(n_trees)]
        self.classes = classes
    
    def train(self, features, labels):
        for tree in self.trees:
            boostrap_features, bootstrap_responses = bootstrap_sampling(features, labels)
            tree.train(boostrap_features, bootstrap_responses)

    def predict(self, x):
        # compute the response of the ensemble from the individual responses and return it
        # my code
        predictions = np.array([])
        for tree in self.trees:
            predictions = np.append(predictions, tree.predict(x))
            
        unique_values, value_counts = np.unique(predictions, return_counts= True)
        most_frequent_index = np.argmax(value_counts)
        most_frequent_value = unique_values[most_frequent_index]
        return most_frequent_value

# Evaluation of Regression and Decision Forest

%%html
<div style="color: green; font-weight:bold"> There is nothing to say about these parts as they are just recalls of three functions that was explained above how they have been implemented wrongly </div>

In [11]:
# perform 5-fold cross-validation (see ex01) with responses +1 and -1 (for 3s and 9s)
# using RegressionForest(n_trees=10)
# and comment on your results
# my code
cross_validation_regression(RegressionForest(n_trees= 10), features_39, responses_39, 5)

Mean absolute error:  0.36974558097598453 +-  0.06854206783839878


In [12]:
# perform 5-fold cross-validation with labels 3 and 9
# using DecisionForest(n_trees=10, classes=np.unique(labels))
# and comment on your results
# my code
cross_validation_classification(ClassificationForest(n_trees= 10, classes= np.unique(labels_39)), features_39, labels_39, 5)

Mean error rate:  0.12408675799086757 +- 0.04110793836588192


# Multi-class Classification Forest

In [13]:
# Train DecisionForest(n_trees=10, classes=np.unique(digits.target))
# for all 10 digits simultaneously.
# Compute and plot the confusion matrix after 5-fold cross-validation and comment on your results.
# my code
digits = load_digits()
print(digits.data.shape, digits.target.shape)
 
features_all = digits.data
labels_all = digits.target

(1797, 64) (1797,)


In [14]:
cross_validation_classification(ClassificationForest(n_trees= 10, classes= np.unique(labels_all)), features_all, labels_all, 5)

KeyboardInterrupt: 

%%html
<div style="color: green; font-weight:bold"> The last two sections of the code are not implemented at all or correctly, they should be just repalced by the working example and doing some modifications to work with this code. Some parts of the code should be revised completely to be identical to the proper answer of the exercise </div>

In [15]:
from sklearn.metrics import confusion_matrix

classes = np.unique(labels_all)
num_sample = 5
k_folds = KFold(n_splits= num_sample)
model = ClassificationForest(classes= np.unique(labels_all), n_trees= 10)
confusion_matrix_total = np.zeros((len(classes), len(classes)))

for i, (train, test) in enumerate(k_folds.split(features_all)):
    model.train(features_all[train], labels_all[train])
    predictions = np.array([])
        
    for f in features_all[test]:
        predictions = np.append(predictions, model.predict(f))
        confusion_matrix_total += confusion_matrix(labels_all[test],predictions, labels=classes)
average_confusion_matrix = confusion_matrix_total/num_sample

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(8, 8))
im = ax.imshow(average_confusion_matrix, cmap='hot')

cbar = ax.figure.colorbar(im, ax=ax)

ax.set_xticks(np.arange(len(classes)))
ax.set_yticks(np.arange(len(classes)))
ax.set_xticklabels(classes)
ax.set_yticklabels(classes)

plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")

for i in range(len(classes)):
    for j in range(len(classes)):
        text = ax.text(j, i, int(average_confusion_matrix[i, j]),
                       ha="center", va="center", color="w")

ax.set_title("Confusion Matrix")
fig.tight_layout()
plt.show()

# One-against-the-rest classification with RegressionForest

In [ ]:
# Train ten one-against-the-rest regression forests for the 10 digits.
# Make sure that all training sets are balanced between the current digit and the rest.
# Assign test instances to the digit with highest score, 
# or to "unknown" if all scores are negative.
# Compute and plot the confusion matrix after 5-fold cross-validation and comment on your results.
... # your code here

Ellipsis